# ZeroShot (Algo 1)

    for total_batches in range(total_n_pseudo_batches):
    
        z = tf.random.normal([batch_size, z_dim])
        pseudo_images = get_gen_images(z)
        teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images, mode=0)

        #generator training
        for ng in range(ng_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            generator_loss = generator_loss(teacher_logits, student_logits)

            #################################
            # BACK PROP AND tick schedulers #
            #################################  

        for ns in range(ns_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            student_loss = student_loss(teacher_logits, teacher_activations, 
                                        student_logits, student_activations, attn_beta)

            #################################
            # BACK PROP AND tick schedulers #
            #################################   

        ######################################################
        ### Val accuracy computation and best model saving ###
        ######################################################    

In [1]:
import tensorflow as tf
from net.generator import NavieGenerator
from utils.cosine_anealing import CosineAnnealingScheduler
from utils.losses import *
#from utils.losses import student_loss_fn
from tensorflow.keras.optimizers import Adam
from net.wide_resnet import WideResidualNetwork
from tensorflow.keras.experimental import CosineDecay
import numpy as np

In [2]:
tf.__version__

'2.0.0'

In [3]:
z_dim = 100
batch_size = 128
ng_batches = 1
ns_batches = 5
attn_beta = 250
total_n_pseudo_batches = 3
n_generator_items = ng_batches + ns_batches
student_lr = 2e-3
generator_lr = 1e-3
number_of_batches = 3

teacher = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0, output_activations=True)
teacher.load_weights('saved_models/cifar10_WRN-16-1_model.005.h5')
teacher.trainable = False

student = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0, output_activations=True)
student_optimizer = Adam(learning_rate=CosineDecay(student_lr, number_of_batches))

generator = NavieGenerator(input_dim=100)
generator_optimizer = Adam(learning_rate=CosineDecay(generator_lr, number_of_batches))

# Generator loss metrics
g_loss_met = tf.keras.metrics.Mean()
# Student loss metrics
stu_loss_met = tf.keras.metrics.Mean()

In [4]:
for total_batches in range(total_n_pseudo_batches):
    # sample from latern space to make an image
    z = tf.random.normal([batch_size, z_dim])

    # Generator training
    generator.trainable = True
    student.trainable = False
    for ng in range(ng_batches):
        with tf.GradientTape() as tape:
            pseudo_imgs = generator(z)
            t_logits, *_ = teacher(pseudo_imgs)
            s_logits, *_ = student(pseudo_imgs)

            # calculate the generator loss
            loss = generator_loss_fn(t_logits, s_logits)

        # The grad for generator
        grads = tape.gradient(loss, generator.trainable_weights)

        # update the generator paramter with the gradient
        generator_optimizer.apply_gradients(zip(grads, generator.trainable_weights))

        g_loss_met(loss)

        print('step %s: generator mean loss = %s' % (total_batches, g_loss_met.result().numpy()))
    # ==========================================================================

    # Student training
    generator.trainable = False
    student.trainable = True
    for ns in range(ns_batches):

        t_logits, *t_acts = teacher(pseudo_imgs)
        with tf.GradientTape() as tape:
            s_logits, *s_acts = student(pseudo_imgs)
            loss = student_loss_fn(t_logits, t_acts, s_logits, s_acts, attn_beta)

        # The grad for student
        grads = tape.gradient(loss, student.trainable_weights)

        # Apply grad for student
        student_optimizer.apply_gradients(zip(grads, student.trainable_weights))

        stu_loss_met(loss)

        print('step %s-%s: studnt mean loss = %s' % (total_batches, ns, stu_loss_met.result().numpy()))


step 0: generator mean loss = -1.5264208
step 0-0: studnt mean loss = 1.5345669
step 0-1: studnt mean loss = 1.4271271
step 0-2: studnt mean loss = 1.2087471
step 0-3: studnt mean loss = 1.0335943
step 0-4: studnt mean loss = 0.92850244
step 1: generator mean loss = -0.92834574
step 1-0: studnt mean loss = 0.8294286
step 1-1: studnt mean loss = 0.75866157
step 1-2: studnt mean loss = 0.70558625
step 1-3: studnt mean loss = 0.66430545
step 1-4: studnt mean loss = 0.63128084
step 2: generator mean loss = -0.70232373
step 2-0: studnt mean loss = 0.59703016
step 2-1: studnt mean loss = 0.56848794
step 2-2: studnt mean loss = 0.5443368
step 2-3: studnt mean loss = 0.5236358
step 2-4: studnt mean loss = 0.505695


In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
from utils.preprocess import load_cifar10_data

In [7]:
(x_train, y_train), (x_test, y_test) = load_cifar10_data()

In [34]:
def get_accuracy(student):
    model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
    model.set_weights(student.get_weights()) 
    model.compile(loss='categorical_crossentropy',
                      metrics=['accuracy'])
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    return loss, accuracy

In [29]:
stest = student(x_test)

TypeError: call() got an unexpected keyword argument 'batch_size'

In [26]:
import timeit

In [35]:
%%time
get_accuracy(student)

Wall time: 7.66 s


(29.924496099853517, 0.1)

In [ ]:
beta

In [13]:
a = True
b = False

In [17]:
if a and not b:
    print("yes")

yes


In [33]:
%%time
model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
model.set_weights(student.get_weights()) 
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

Wall time: 577 ms


In [31]:
%%time
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 2.3115152214050294
Test accuracy: 0.1098
Wall time: 7.44 s


In [ ]:
student.save()